# Bilinear interpolation of coarse precip

In [ ]:
%reload_ext autoreload

%autoreload 2

%reload_ext dotenv
%dotenv

import math
import os

import iris
import iris.analysis.cartography
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from mlde_utils import VariableMetadata, platecarree, cp_model_rotated_pole
from mlde_utils.utils import si_to_mmday
from mlde_notebooks import create_map_fig, plot_map
from mlde_data.preprocessing.regrid import Regrid

## CPM

In [ ]:
ds_meta = VariableMetadata(
    os.path.join(os.getenv("DERIVED_DATA"), "moose"),
    variable="linpr", domain="birmingham-64", frequency="day", resolution="2.2km-coarsened-gcm-2.2km-coarsened-4x", ensemble_member="01"
)

ds = xr.open_mfdataset(ds_meta.existing_filepaths())
ds["linpr"] = si_to_mmday(ds, "linpr")
ds

In [ ]:
ds["linpr"].plot(bins=100)

In [ ]:
fig = plt.figure(layout="constrained")

fig, axd = create_map_fig([["min"]])

plot_map(ds["linpr"].min(dim=["time"]), ax=axd["min"], style=None, add_colorbar=True)

## GCM

In [ ]:
ds_meta = VariableMetadata(
    os.path.join(os.getenv("DERIVED_DATA"), "moose"),
    variable="linpr", domain="birmingham-64", frequency="day", resolution="60km-2.2km-coarsened-4x", ensemble_member="01"
)

ds = xr.open_mfdataset(ds_meta.existing_filepaths())
ds["linpr"] = si_to_mmday(ds, "linpr")
ds

In [ ]:
ds["linpr"].plot(bins=100)

In [ ]:
fig = plt.figure(layout="constrained")

fig, axd = create_map_fig([["min"]])

plot_map(ds["linpr"].min(dim=["time"]), ax=axd["min"], style=None, add_colorbar=True)

Negative values occur on the Greenwich Meridian this confirms both that the interpolating is happening on a grid split on 0 longitude (rather than at the edge of the domain) and that this is the cause of the very low numbers

In [ ]:
ds["linpr"].isel(time=5).min().values

In [ ]:
ds["linpr"].isel(time=5).time

## GCM precip

In [ ]:
ds_meta = VariableMetadata(
    os.path.join(os.getenv("MOOSE_DATA")),
    variable="pr", domain="global", frequency="day", resolution="60km", ensemble_member="01"
)
ds = xr.open_dataset(ds_meta.filepath(1981))

ds["pr"] = si_to_mmday(ds, "precipitation_flux")
ds

In [ ]:
ds["pr"].sel(time="1981-05-30")

In [ ]:
lon_name = "longitude"

lon_attrs = ds[lon_name].attrs
ds.coords[lon_name] = (ds.coords[lon_name] + 180) % 360 - 180
ds = ds.sortby(ds[lon_name])
ds[lon_name].attrs =lon_attrs

ds["pr"].sel(time="1981-05-30")

In [ ]:
ds[lon_name].attrs =lon_attrs
ds["pr"].sel(time="1981-05-30")

In [ ]:
subplot_kw = dict(projection=platecarree, transform=platecarree)
fig = plt.figure(layout="constrained")
ax = fig.add_subplot(111, **subplot_kw)

plot_map(ds["pr"].sel(time="1981-05-30").squeeze(), ax=ax, style=None, add_colorbar=True)

In [ ]:
target_grid_filepath = os.path.join("~/cdt/summer/code",
    "mlde-data/src/mlde_data/target-grids/2.2km-coarsened-4x/uk",
    "moose_grid.nc",
)

subplot_kw = dict(projection=cp_model_rotated_pole)
fig = plt.figure(layout="constrained")
ax = fig.add_subplot(111, **subplot_kw)

# ds["longitude"] = ds["longitude"]-180.0
ds
plot_map(Regrid(target_grid_filepath, variables=["pr"], scheme="linear").run(ds)["pr"].min(dim=["time"]), ax=ax, style=None, add_colorbar=True)

In [ ]:
precip_cube = ds["pr"].to_iris()
precip_cube

In [ ]:
precip_cube.coords(axis="x")[0].ndim

In [ ]:
lon_name = "longitude"

ds.coords[lon_name] = (ds.coords[lon_name] + 180) % 360 - 180
ds = ds.sortby(ds[lon_name])
ds["pr"].sel(time="1981-05-30")
ds
# precip_cube = ds["pr"].to_iris()
# precip_cube

In [ ]:
precip_cube.coords(axis="x")